In [3]:
import numpy as np
from scipy import linalg
from scipy.sparse.linalg import eigsh
from scipy import sparse

hamiltonian = np.zeros((6240,6240))

with open('datafile','r') as data:
    count = 0
    while True:
        line = data.readline()
        if count == 0:
            count += 1
        if not line:
            break
        else:
            values = line.split()
            hamiltonian[int(values[0])-1][int(values[1])-1] = float(values[2])
            hamiltonian[int(values[1])-1][int(values[0])-1] = float(values[2])
            

sparse_hamiltonian = sparse.csr_matrix(hamiltonian)
eigenvals, eigenvecs = eigsh(sparse_hamiltonian,1,which='LM')

In [4]:
sub_hamiltonian = hamiltonian[0:100,0:100] # used 100 particles to work with

In [5]:
sparse_sub_hamiltonian = sparse.csr_matrix(sub_hamiltonian)
eigenvalues, eigenvectors = eigsh(sparse_sub_hamiltonian,99,which = 'LM')
print sum(eigenvalues)

-8234.4466281


In [6]:
# arbitrarily set one of the determinants to 1.
psi_initial = np.zeros(shape=(100, 1), dtype = complex)
psi_initial[0] = 1 # I set the first determinant to 1

In [7]:
# Without Runge-Kutta implemented and normalized
dt = 0.01
psi_prev = psi_initial
psi_next = np.zeros(shape=(100,1),dtype = complex)
for t in range(10):
    for i in range(100):
        accumulator = sub_hamiltonian[i][i]*psi_prev[i]
        for j in range(100):
            if i != j:
                accumulator += sub_hamiltonian[i][j]*psi_prev[j]
        psi_next[i] = accumulator*dt*1j
        accumulator = 0 # reset accumulator to 0
    psi_prev = psi_next/np.sqrt(sum((np.absolute(psi_next)**2))) # normalized the coefficients
    psi_next = np.zeros(shape=(100, 1),dtype=complex) # reset to 0s

In [8]:
print sum((np.absolute(psi_prev)**2))

[ 1.]


In [9]:
# Without Runge-Kutta implemented and not normalized (for comparison)
dt = 0.01
psi_prev = psi_initial
psi_next = np.zeros(shape=(100,1),dtype = complex)
for t in range(10):
    for i in range(100):
        accumulator = sub_hamiltonian[i][i]*psi_prev[i]
        for j in range(100):
            if i != j:
                accumulator += sub_hamiltonian[i][j]*psi_prev[j]
        psi_next[i] = accumulator*dt*1j
        accumulator = 0 # reset accumulator to 0
    psi_prev = psi_next # coefficients not normalized
    psi_next = np.zeros(shape=(100, 1),dtype=complex) # reset to 0s

In [10]:
print sum((np.absolute(psi_prev)**2))

[ 0.15181654]


In [19]:
# Without Runge-Kutta implemented and not normalized (different time steps)
dt = 0.01
psi_prev = psi_initial
psi_next = np.zeros(shape=(100,1),dtype = complex)
time_steps = list()
probabilities = list()
for time_step in range(10):
    dt = (10**time_step)*(10**(-5))
    time_steps.append(dt)
    psi_prev = psi_initial
    psi_next = np.zeros(shape=(100,1),dtype = complex)
    for t in range(10):
        for i in range(100):
            accumulator = sub_hamiltonian[i][i]*psi_prev[i]
            for j in range(100):
                if i != j:
                    accumulator += sub_hamiltonian[i][j]*psi_prev[j]
            psi_next[i] = accumulator*dt*1j
            accumulator = 0 # reset accumulator to 0
        psi_prev = psi_next # coefficients not normalized
        psi_next = np.zeros(shape=(100, 1),dtype=complex) # reset to 0s
    val = sum(np.absolute(psi_prev)**2)
    probabilities.append(val[0])
print probabilities

[1.5181653973459046e-61, 1.5181653973459063e-41, 1.5181653973459027e-21, 0.15181653973459028, 1.5181653973459065e+19, 1.5181653973459046e+39, 1.5181653973459118e+59, 1.5181653973459132e+79, 1.5181653973459108e+99, 1.5181653973459089e+119]


In [ ]:
# this graph plots the divergence in the sum of probabilities for the 4th-Order Runge Kutta
import matplotlib.pyplot as plt
import math 
logged_time = list()
logged_probabilities = list()
constant = list()
for t in range(len(time_steps)):
    logged_time.append(math.log(time_steps[t]))
    logged_probabilities.append(math.log(probabilities[t]))
x = np.arange(-5, max(logged_time)-5)
constant = x*0 + 1
plt.plot(x,logged_probabilities,label='Sum of probabilities')
# plt.plot(time_steps,probabilities) # comment this out
plt.plot(x, constant, label = 'Normalization value of 1')
plt.xlabel("Log of time steps", fontsize=15)
plt.ylabel("Log of sum of Probabilities after 10 time steps", fontsize =15)
plt.legend(loc='upper left')
plt.show()

In [39]:
#Implement 4th order Runge Kutta
# accumulator variable below refers to RHS of equation 2 in write-up
# Use generalized nth order runge-kutta? Based on the error of each time step
dt = 0.01
psi_prev_rk = psi_initial
psi_next = np.zeros(shape=(100,1),dtype = complex)
k1 = np.zeros(shape=(100,1),dtype = complex)
k2 = np.zeros(shape=(100,1),dtype = complex)
k3 = np.zeros(shape=(100,1),dtype = complex)
k4 = np.zeros(shape=(100,1),dtype = complex)
y1 = np.zeros(shape=(100,1),dtype = complex)
y2 = np.zeros(shape=(100,1),dtype = complex)
y3 = np.zeros(shape=(100,1),dtype = complex)
for t in range(10): # wrong time step
    # first loop to update k1, y1
    for i in range(100):
        accumulator = sub_hamiltonian[i][i]*psi_prev_rk[i]
        for j in range(100):
            if i != j:
                accumulator += sub_hamiltonian[i][j]*psi_prev_rk[j]
        k1[i] = accumulator*1j
        y1[i] = psi_prev_rk[i] + accumulator*(dt/2)*1j
        accumulator = 0 # reset accumulator to 0
    # To get k2, y2
    assert accumulator == 0
    for i in range(100):
        accumulator = sub_hamiltonian[i][i]*y1[i]
        for j in range(100):
            if i != j:
                accumulator += sub_hamiltonian[i][j]*y1[j]
        k2[i] = accumulator*1j
        y2[i] = psi_prev_rk[i] + accumulator*(dt/2)*1j
        accumulator = 0 # reset accumulator to 0
    # To get k3, y3
    assert accumulator == 0
    for i in range(100):
        accumulator = sub_hamiltonian[i][i]*y2[i]
        for j in range(100):
            if i != j:
                accumulator += sub_hamiltonian[i][j]*y2[j]
        k3[i] = accumulator*1j
        y3[i] = psi_prev_rk[i] + accumulator*dt*1j
        accumulator = 0 # reset accumulator to 0
    # To get k4, y4
    assert accumulator == 0
    for i in range(100):
        accumulator = sub_hamiltonian[i][i]*y3[i]
        for j in range(100):
            if i != j:
                accumulator += sub_hamiltonian[i][j]*y3[j]
        k4[i] = accumulator*1j
        accumulator = 0 # reset accumulator to 0
        psi_next[i] = psi_prev_rk[i] + (k1[i]+2*k2[i]+2*k3[i]+k4[i])*(dt/6)
    psi_prev_rk = psi_next 
    psi_next = np.zeros(shape=(100,1),dtype = complex)

In [327]:
print sum((np.absolute(psi_prev_rk)**2)) # possibility of using lost probabilities as indicator of how good the approximation is?

[ 0.93151081]


In [88]:
# # verlet algorithm (part 1 - initializing variables)
# dt = 0.01
# psi_t_minus_dt = psi_initial
# psi_t = np.zeros(shape=(100,1),dtype = complex)
# psi_t_minus_derivative = np.zeros(shape=(100,1),dtype = complex)

# # Get r(t + dt) first. Maybe I should use the Runge-Kutta here instead?
# # for i in range(100):
# #     accumulator = sub_hamiltonian[i][i]*psi_t_minus_dt[i]
# #     for j in range(100):
# #         if i != j:
# #             accumulator += sub_hamiltonian[i][j]*psi_t_minus_dt[j]
# #     psi_t[i] = accumulator*dt*1j
# #     accumulator = 0 # reset accumulator to 0
# # psi_t = psi_t#/np.sqrt(sum((np.absolute(psi_t)**2)))
# # psi_t_minus_derivative = (psi_t - psi_t_minus_dt)/dt

# # Use Runge-Kutta instead to initialize the first r(t)
# for i in range(100):
#     accumulator = sub_hamiltonian[i][i]*psi_t_minus_dt[i]
#     for j in range(100):
#         if i != j:
#             accumulator += sub_hamiltonian[i][j]*psi_t_minus_dt[j]
#     k1[i] = accumulator*1j
#     y1[i] = psi_t_minus_dt[i] + accumulator*(dt/2)*1j
#     accumulator = 0 # reset accumulator to 0
# # To get k2, y2
# assert accumulator == 0
# for i in range(100):
#     accumulator = sub_hamiltonian[i][i]*y1[i]
#     for j in range(100):
#         if i != j:
#             accumulator += sub_hamiltonian[i][j]*y1[j]
#     k2[i] = accumulator*1j
#     y2[i] = psi_t_minus_dt[i] + accumulator*(dt/2)*1j
#     accumulator = 0 # reset accumulator to 0
# # To get k3, y3
# assert accumulator == 0
# for i in range(100):
#     accumulator = sub_hamiltonian[i][i]*y2[i]
#     for j in range(100):
#         if i != j:
#             accumulator += sub_hamiltonian[i][j]*y2[j]
#     k3[i] = accumulator*1j
#     y3[i] = psi_t_minus_dt[i] + accumulator*dt*1j
#     accumulator = 0 # reset accumulator to 0
# # To get k4, y4
# assert accumulator == 0
# for i in range(100):
#     accumulator = sub_hamiltonian[i][i]*y3[i]
#     for j in range(100):
#         if i != j:
#             accumulator += sub_hamiltonian[i][j]*y3[j]
#     k4[i] = accumulator*1j
#     accumulator = 0 # reset accumulator to 0
#     psi_t[i] = psi_t_minus_dt[i] + (k1[i]+2*k2[i]+2*k3[i]+k4[i])*(dt/6)
# psi_t_minus_derivative = (k1 + 2*k2+ 2*k3+ k4)*(1.0/6) # why is dt required here?
# print sum(np.absolute(psi_t_minus_derivative)**2)

[ 7638.16758144]


In [89]:
# print sum((np.absolute(psi_t)**2))

[ 0.99293027]


In [90]:
# psi_t = psi_t/np.sqrt(sum(np.absolute(psi_t)**2)) # remove this if it compromises on accuracy

In [96]:
# verlet algorithm (part 1 - initializing variables)
dt = 0.01
psi_t_minus_dt = psi_initial
psi_t = np.zeros(shape=(100,1),dtype = complex)
psi_t_minus_derivative = np.zeros(shape=(100,1),dtype = complex)

# Get r(t + dt) first. Maybe I should use the Runge-Kutta here instead?
# for i in range(100):
#     accumulator = sub_hamiltonian[i][i]*psi_t_minus_dt[i]
#     for j in range(100):
#         if i != j:
#             accumulator += sub_hamiltonian[i][j]*psi_t_minus_dt[j]
#     psi_t[i] = accumulator*dt*1j
#     accumulator = 0 # reset accumulator to 0
# psi_t = psi_t#/np.sqrt(sum((np.absolute(psi_t)**2)))
# psi_t_minus_derivative = (psi_t - psi_t_minus_dt)/dt

# Use Runge-Kutta instead to initialize the first r(t)
for i in range(100):
    accumulator = sub_hamiltonian[i][i]*psi_t_minus_dt[i]
    for j in range(100):
        if i != j:
            accumulator += sub_hamiltonian[i][j]*psi_t_minus_dt[j]
    k1[i] = accumulator*1j
    y1[i] = psi_t_minus_dt[i] + accumulator*(dt/2)*1j
    accumulator = 0 # reset accumulator to 0
# To get k2, y2
assert accumulator == 0
for i in range(100):
    accumulator = sub_hamiltonian[i][i]*y1[i]
    for j in range(100):
        if i != j:
            accumulator += sub_hamiltonian[i][j]*y1[j]
    k2[i] = accumulator*1j
    y2[i] = psi_t_minus_dt[i] + accumulator*(dt/2)*1j
    accumulator = 0 # reset accumulator to 0
# To get k3, y3
assert accumulator == 0
for i in range(100):
    accumulator = sub_hamiltonian[i][i]*y2[i]
    for j in range(100):
        if i != j:
            accumulator += sub_hamiltonian[i][j]*y2[j]
    k3[i] = accumulator*1j
    y3[i] = psi_t_minus_dt[i] + accumulator*dt*1j
    accumulator = 0 # reset accumulator to 0
# To get k4, y4
assert accumulator == 0
for i in range(100):
    accumulator = sub_hamiltonian[i][i]*y3[i]
    for j in range(100):
        if i != j:
            accumulator += sub_hamiltonian[i][j]*y3[j]
    k4[i] = accumulator*1j
    accumulator = 0 # reset accumulator to 0
    psi_t[i] = psi_t_minus_dt[i] + (k1[i]+2*k2[i]+2*k3[i]+k4[i])*(dt/6)
psi_t_minus_derivative = (k1 + 2*k2+ 2*k3+ k4)*(1.0/6) # why is dt required here?
# print sum(np.absolute(psi_t_minus_derivative)**2)

# verlet algorithm part 2. Combine with above method (Do not run this by itself-- update of variables needed)
psi_t_plus_derivative = np.zeros(shape=(100,1),dtype = complex)
psi_t_plus_dt = np.zeros(shape=(100,1),dtype = complex)

number_steps = 10
dt = 0.00000001
for t in range(number_steps):
    for i in range(100):
        accumulator = sub_hamiltonian[i][i]*psi_t[i]
        for j in range(100):
            if i != j:
                accumulator += sub_hamiltonian[i][j]*psi_t[j]
        k1[i] = accumulator*1j
        y1[i] = psi_t[i] + accumulator*(dt/2)*1j
        accumulator = 0 # reset accumulator to 0
    # To get k2, y2
    assert accumulator == 0
    for i in range(100):
        accumulator = sub_hamiltonian[i][i]*y1[i]
        for j in range(100):
            if i != j:
                accumulator += sub_hamiltonian[i][j]*y1[j]
        k2[i] = accumulator*1j
        y2[i] = psi_t[i] + accumulator*(dt/2)*1j
        accumulator = 0 # reset accumulator to 0
    # To get k3, y3
    assert accumulator == 0
    for i in range(100):
        accumulator = sub_hamiltonian[i][i]*y2[i]
        for j in range(100):
            if i != j:
                accumulator += sub_hamiltonian[i][j]*y2[j]
        k3[i] = accumulator*1j
        y3[i] = psi_t[i] + accumulator*dt*1j
        accumulator = 0 # reset accumulator to 0
    # To get k4, y4
    assert accumulator == 0
    for i in range(100):
        accumulator = sub_hamiltonian[i][i]*y3[i]
        for j in range(100):
            if i != j:
                accumulator += sub_hamiltonian[i][j]*y3[j]
        k4[i] = accumulator*1j
        accumulator = 0 # reset accumulator to 0
        psi_t_plus_derivative[i] = (k1[i]+2*k2[i]+2*k3[i]+k4[i])*(1.0/6)
        psi_t_plus_dt[i] = 2*psi_t[i] - psi_t_minus_dt[i] + (psi_t_plus_derivative[i] - psi_t_minus_derivative[i])*(dt*dt)
    psi_minus_dt = psi_t
    psi_t = psi_t_plus_dt
    psi_t_plus_dt = np.zeros(shape=(100,1),dtype = complex)
    psi_t_minus_derivative = psi_t_plus_derivative
    print sum((np.absolute(psi_t)**2))

[ 2.51349406]
[ 10.13752219]
[ 43.71718063]
[ 184.20290631]
[ 758.47999278]
[ 3080.25670624]
[ 12416.7002952]
[ 49861.14812127]
[ 199836.28636599]
[ 800131.53322578]


In [ ]:
x = [1,2,3,4,5,6,7,8,9,10]
plt.plot(x,[2.513,10.137,43.717,184.20,758.47,3080.25,12416.70,49861.14,199836.28,800131.53])
plt.show()